참고문헌 : Pytel 외 저, 이주성 외 역, 재료역학, 2판, 한티미디어, 2013.<br>Ref: Pytel, Kiusalaas, Sharma, Mechanics of Materials, 2nd Ed., Cengage Learning, 2013.

`python` 기능을 확장해 주는 `module`을 불러 들임 (일부 기능만 사용될 수도 있음)<br>
Bring in `module`'s that would expand features of `python`. (This file may use just some of them.)

In [ ]:
import numpy as np  # 배열, 행렬 관련 기능
import numpy.linalg as na  # 선형대수 (벡터, 행렬) 관련 기능
import matplotlib.pyplot as plt  # 그래프 관련 기능
import scipy.integrate as si  # 적분 관련 기능
import sympy as sy  # 기호 연산 기능
import sympy.plotting as splot
import IPython.display as disp  # 웹페이지 표시 기능
sy.init_printing()  # 기호 연산 결과 표시 기능 준비

## 예제 06.011<br>ex06.011

p. 281

### 문제에서 주어진 변수<br>Given Parameters<br>Given variables

#### 보의 길이<br>Length of the beam

In [ ]:
L_AB_m, L_BC_m, L_CD_m = sy.symbols('L_{AB}[m], L_{BC}[m], L_{CD}[m]', positive=True)
L_AC_m = L_AB_m + L_BC_m
L_AD_m = L_AC_m + L_CD_m

s_d = {
    L_AB_m: 2, 
    L_BC_m: 4, 
    L_CD_m: 3
}

#### 하중<br>Load

In [ ]:
P_B_N, P_C_N = sy.symbols('P_{B}[N], P_{C}[N]', real=True)

s_d.update({
        P_B_N: -100,
        P_C_N: -80
    }
)

#### 관심 지점<br>Point of interest

In [ ]:
x_m = sy.symbols('x[m]', real=True)
x_center_m = L_AD_m / 2

### 처짐 공식<br>Deflection formula

p. 280 표 6.3<br>Table 6.3 p.280

$$
\delta = \begin{cases}
    \frac{Pbx}{6LEI}\left( L^2 - x^2 - b^2 \right) & 0 \le x \le a \\
    \frac{Pb}{6LEI}\left[ \frac{L}{b} \left(x-a\right)^3 + \left(L^2-b^2\right)x - x^3 \right] & a \le x \le L \\
\end{cases}
$$

변수 정의<br>Define variables

In [ ]:
P_N = sy.Symbol('P[N]', real=True)
a_m = sy.Symbol('a[m]', negative=False, real=True)
L_m, E_Pa, I_m4 = sy.symbols('L[m], E[Pa], I[m^{4}]', positive=True)
b_m = L_m - a_m

하중의 오른쪽의 처짐<br>Deflection of right side of the load

In [ ]:
EI_delta_right_Nm3 = (P_N * b_m / (6 * L_m * E_Pa * I_m4)) * ((L_m/b_m)*(x_m - a_m)**3 + x_m * (L_m**2 - b_m**2) - x_m **3)

$x=L$ 지점에서 0인지 확인<br>Check if $\delta=0$ at $x=L$

In [ ]:
assert(0 == sy.simplify(EI_delta_right_Nm3.subs(x_m, L_m)))

하중의 왼쪽의 처짐<br>Deflection of left side of the load

In [ ]:
EI_delta_left_Nm3 = (P_N * b_m * x_m / (6 * L_m * E_Pa * I_m4)) * (L_m**2 - x_m **2 - b_m**2)

In [ ]:
assert(0 == sy.simplify(EI_delta_left_Nm3.subs(x_m, 0)))

하중 지점에서 연속인지 확인<br>Check if continuous at the point of the load

In [ ]:
assert(0 == sy.simplify(EI_delta_left_Nm3.subs(x_m, a_m) - EI_delta_right_Nm3.subs(x_m, a_m)))

In [ ]:
EI_delta_Nm3 = EI_delta_left_Nm3 + (EI_delta_right_Nm3 - EI_delta_left_Nm3) * sy.Heaviside(x_m - a_m)

$x=0$ 지점에서 0인지 확인<br>Check if $\delta=0$ at $x=0$

In [ ]:
assert(0 == sy.simplify(EI_delta_Nm3.subs(a_m, L_AB_m).subs(x_m, 0).subs(s_d)))

#### B점 하중<br>Load at B

In [ ]:
EI_delta_b_Nm3 = sy.simplify(
    EI_delta_Nm3.subs(
        {
            L_m: L_AD_m,
            a_m: L_AB_m,
            P_N: P_B_N,
        }
    )
)

변수값을 적용<br>Substitute values

In [ ]:
EI_delta_b_Nm3.subs(s_d)

In [ ]:
splot.plot(EI_delta_b_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}), (x_m, 0, L_AD_m.subs(s_d)), xlabel='x(m)', ylabel='$EI\\delta(Nm^3)$')

$\delta_{x=0}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_b_Nm3.subs(s_d).subs(x_m, 0))

$\delta_{x=L}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_b_Nm3.subs(x_m, L_AD_m).subs(s_d))

#### C점 하중<br>Load at C

In [ ]:
EI_delta_c_Nm3 = sy.simplify(
    EI_delta_Nm3.subs(
        {
            L_m: L_AD_m,
            a_m: L_AC_m,
            P_N: P_C_N,
        }
    )
)

변수값을 적용<br>Substitute values

In [ ]:
EI_delta_c_Nm3.subs(s_d)

In [ ]:
splot.plot(EI_delta_c_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}), (x_m, 0, L_AD_m.subs(s_d)), xlabel='x(m)', ylabel='$EI\\delta(Nm^3)$')

$\delta_{x=0}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_c_Nm3.subs(s_d).subs(x_m, 0))

$\delta_{x=L}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_c_Nm3.subs(x_m, L_AD_m).subs(s_d))

#### 중첩<br>Superposition

In [ ]:
EI_delta_b_c_Nm3 = sy.simplify(EI_delta_b_Nm3 + EI_delta_c_Nm3)

변수값을 적용<br>Substitute values

In [ ]:
sy.simplify(EI_delta_b_c_Nm3.subs(s_d))

위 네 식은 각각 C~D, B~C, 불가능한 영역, A~B 구간에 해당한다.<br>The equations above are for C~D, B~C, impossible, and A~B intervals.

In [ ]:
splot.plot(
    EI_delta_b_c_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}), 
    EI_delta_b_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}),
    EI_delta_c_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}),
    (x_m, 0, L_AD_m.subs(s_d)), xlabel='x(m)', ylabel='$EI\\delta(Nm^3)$'
)


$\delta_{x=0}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_b_c_Nm3.subs(s_d).subs(x_m, 0))

$\delta_{x=L}=0?$

In [ ]:
assert 1e-6 > abs(EI_delta_b_c_Nm3.subs(x_m, L_AD_m).subs(s_d))

#### 관심 지점에서의 처짐<br>Deflection at the Point of interest

In [ ]:
EI_delta_center_Nm3 = sy.simplify(EI_delta_b_c_Nm3.subs(x_m, L_AD_m/2))

In [ ]:
EI_delta_center_Nm3

In [ ]:
EI_delta_center_Nm3.subs(s_d)

In [ ]:
float(EI_delta_center_Nm3.subs(s_d).subs({E_Pa:1, I_m4:1}))